In [ ]:
import numpy as np
import scipy.integrate as integrate
from numpy.linalg import inv
import math as m
from functions import *
import importlib

%load_ext autoreload

%autoreload 2

np.set_printoptions(suppress=True)

This next block will create a class for all of our inputs to be placed into. This will allow for all inputs to have easily accessed names, values, and units.

In [ ]:
class Input:
    def __init__(self, name, value, unit):
        self.name = name
        self.value = value
        #self.valueType = type(value)
        self.unit = unit
    
    def printValues(self):
        print("\n")
        for attr, value in self.__dict__.items():
            print(f"{attr}: {value}")

# Please fill in all inputs and their units below
# v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v 

For Reference: Units Set: MPa, mm2, mm, N

In [ ]:
YoungsModulus = 210000
YM_Unit = "MPa"

Area = 200
A_Unit = "mm2"


# node connections for each element
nArray = np.array([[1,2], [2,3], [3,7], [4,5], [5,6], [6,7], [1,4], [2,5], [3,6], [1,5], [2,6]])
print(f"number of elements: {len(nArray)}")

coords = np.array([[0,0], [4,0], [7,0], [0,2.5], [4,4], [7,4], [8.5,3]])

# list of lengths corresponding to each element
# the length of element n is contained in lengths



# list of angles relative to positive x axis for each element
#aArray = np.array([(7/6), 1, 0.5, (5/6), 1, (5/6), 0.5])
#aArray *= m.pi

Angle_unit = "rad"

# input boundary conditions below
# for displacements: [node#, u, v]
# for forces: [node#, Fx, Fy]

uv_BCs = np.array([[3, 0, 0],[4, 0, None]])

force_BCs = np.array([[4, 0, 3000], [5, 0, -2000], [6, 0, 2000], [7, 0, -5000]])

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
print(range(len(coords)))


In [ ]:
E = Input("Young's Modulus", YoungsModulus, YM_Unit)

A = Input("Area", Area, A_Unit)

nodes = Input("Node numbers", nArray, "None")
nodes.printValues()



nodeCoordinates = Input("Node Coordinates (x, y)", coords, "None")
nodeCoordinates.printValues()

number of elements is the number of rows in nodes
<br>number of nodes is the maximum number found in nodes

In [ ]:
numElems = len(nodes.value)
print(f"number of elements: {numElems}")
numNodes = nodes.value.max()
print(f"number of nodes: {numNodes}")

In [ ]:
length = np.zeros(numElems)
Length_unit = "mm"
print(length)

angle = np.zeros(numElems)
print(len(angle))

for i in range(numElems):
    connection = nodes.value[i]
    N1 = connection[0]
    N2 = connection[1]
    x1 = (nodeCoordinates.value[N1-1])[0]
    y1 = (nodeCoordinates.value[N1-1])[1]

    x2 = (nodeCoordinates.value[N2-1])[0]
    y2 = (nodeCoordinates.value[N2-1])[1]
    length[i] = getTrussLength(x1, y1, x2, y2)
    opposite = y2 - y1
    adjacent = x2 - x1
    angle[i] = m.atan(opposite/adjacent)
    
    print(f"element {i+1}: length = {length[i]} m; angle = {angle[i]}")

length *= 1000
print(length)

print(angle)

lengths = Input("Lengths", length, Length_unit)
lengths.printValues()

angles = Input("Element Angles (relative to the positive x axis)", angle, Angle_unit)
angles.printValues()

two reactions per node
<br>uvList -> list of unknowns (node displacements)
<br>we can input our known values -> fixed_nodes at nodes 4 & 5
<br>these fixed nodes numbers are taken straight from diagram (not adjusted for python counting)
<br><br>u and v for any given node is given as (node# - 1)* 2 and ((node# -1)* 2) + 1 respectively.


In [ ]:
uvList = [None]*(numNodes*2)
print("displacement bcs: [node number, x disp (u), y disp (v)]")
print(uv_BCs)

adjust_array(uvList, uv_BCs, "displacements")
print("u & v list:")
print(uvList)

Same procedure as displacement BCs for forces
<br>Input BCs given as : [node #, x force, y force]

In [ ]:
#forceList = [0]*(numNodes*2)
forceList = np.zeros(((numNodes*2), 1))

nodesWithLoads = []

loadedNodes = 0

for i in range(len(force_BCs)):
    if force_BCs[i,1] or force_BCs[i,2] != None:
        loadedNodes += 1
        nodesWithLoads.append((force_BCs[i,0] - 1))


print(f"number of nodes with concentrated loads: {loadedNodes}")
print(f"at nodes: {nodesWithLoads}")

#adjust force array given concentrated loading conditions
adjust_array(forceList, force_BCs, "forces")

print(f"force list: \n{forceList}")

Global [k] matrix is a symmetric array of shape (numNodes x 2, numNodes x 2)
<br>Bar Constants are one per element, stored in a numElemes * 1 array.

In [ ]:
globalK = np.zeros(((numNodes*2),(numNodes*2)))
print(f"Global K array dimensions: {globalK.shape}")

barConstant = np.zeros((numElems,1))
for i in range(numElems):
    print(f"E = {E.value}, A ={A.value}, length for element {i+1} = {lengths.value[i]}")
    barConstant[i] = (E.value*A.value)/lengths.value[i]

print(f"Bar Constants = \n{barConstant}")

## Creating the finite element equation for each Truss Element
### Combining into the global K array

In [ ]:
for i in range(numElems):
    kMatrix = create_truss_k(angles.value[i])
    kMatrix *= barConstant[i]
    print(f"\nk matrix for element {i+1}: \n{kMatrix}")
    #print(kMatrix)
    startNode = nodes.value[i,0]
    endNode = nodes.value[i,1]
    print(f"Element {i+1} starting at node: {startNode} and ending at node: {endNode}")
    #print(np.vsplit(kMatrix, 2))
    Sn = (startNode - 1)*2
    En = (endNode - 1)*2
    #first, adjust 0,0 -> then, 2,0; 0,2; and 2,2
    block_array_adjust(Sn, Sn, globalK, 0, 0, kMatrix)
    block_array_adjust(En, Sn, globalK, 2, 0, kMatrix)
    block_array_adjust(Sn, En, globalK, 0, 2, kMatrix)
    block_array_adjust(En, En, globalK, 2, 2, kMatrix)

print(f"\nGlobal K matrix: \n{globalK}")
unmodified_globalK = globalK.copy()
check_Symmetric(globalK)

In [ ]:
j = 0
while j < numNodes*2:
    print(f"{j}")
    #if j in nodesWithLoads:
     #   j+=2
    #first case: displacement BC = 0. In this case, set combined force values to 0, and alter 
    #globalK to reflect the changes.
    if uvList[j] == 0:
        print(f"u = 0 at row: {j}")
        forceList[j] = 0 
        globalK[j] = 0
        globalK[j,j] = 1
        j+=1
    elif uvList[j] == None:
        print(f"none detected at row {j}")
        j+=1
    else: #when we have a known, non-zero displacement boundary condition
        #alter global stiffness to all zeros and one 1, and set force equal to displacement BC
        print(f"row {j} altered")
        print(uvList[j])
        forceList[j] = uvList[j]
        for c in range(numNodes):
            if (globalK[j,c] > 0) == True:
                globalK[j,c] = 1
            else:
                globalK[j,c] = 0
        j+=1

                
print(globalK)
print(forceList)

In [ ]:
#globalK.round(0)
globalK_inv = inv(globalK)
#print(globalK_inv)

node = 1

uFinal = np.matmul(globalK_inv, forceList)

for i in range(len(uFinal)):
    if check_even(i) == True:
        print(f"u{node} = {uFinal[i]}")
    else:
        print(f"v{node} = {uFinal[i]}\n")
        node += 1 

Forces can be found from [K]*[u]

In [ ]:
forcesFinal = np.matmul(unmodified_globalK, uFinal)
for i in range(len(forceList)):
    if forceList[i] != forcesFinal[i] and forceList[i] != 0 and forceList[i] != None:
        forcesFinal[i] = forceList[i]
print(f"Final Forces: \n{forcesFinal}")

node = 1

for i in range(len(forcesFinal)):
    if check_even(i) == True:
        print(f"fx{node} = {forcesFinal[i]}")
    else:
        print(f"fy{node} = {forcesFinal[i]}")
        node += 1 


## Finding Stresses

In [ ]:
stressFinal = np.zeros((numElems, 1))
print(stressFinal)
for i in range(numElems):
    startNode = nodes.value[i,0]
    endNode = nodes.value[i,1]
    print(f"Element {i+1} starts at node: {startNode} and ends at node: {endNode}")
    #print(np.vsplit(kMatrix, 2))
    Sn = (startNode - 1)*2
    En = (endNode - 1)*2
    print(f"adjusted start node: {Sn} adjusted end node: {En}")
    trigMatrix = create_truss_stress_trig_matrix(angles.value[i])
    print(f"trigMatrix = \n{trigMatrix}")
    local_uMatrix = np.array([
        uFinal[Sn], 
        uFinal[(Sn + 1)], 
        uFinal[En], 
        uFinal[(En+1)]
        ])
    print(f"local u matrix: \n{local_uMatrix}")
    localLength = np.array([[(-1/lengths.value[i]), 1/lengths.value[i]]])
    print(f"local length matrix: \n{localLength}")

    indStress = (trigMatrix @ local_uMatrix)
    indStress = localLength @ indStress
    indStress *= E.value

    stressFinal[i] = indStress
    
    
    print(f"Stress in element = {indStress}")

    

In [ ]:
print(stressFinal)

In [ ]:
np.set_printoptions(suppress=True, formatter={'float':"{:0.4f}".format})
printTrussNodalDisplacements(uFinal, lengths.unit)
printStresses(stressFinal,E.unit)